# Weed Detection and Classification with Instance Segmentation
**Date:** 2025-04-05  
**Author:** arfaouiahmed1  
**Framework:** YOLOv8 (Ultralytics)  
**Environment:** Windows 11 (System CUDA 12.8), PyTorch with CUDA 12.1  
**Note:** PyTorch was installed using the official command for CUDA 12.1, which determines the CUDA toolkit version PyTorch interacts with.

This notebook implements an instance segmentation model to detect, outline, and classify different types of weeds using the Roboflow dataset: https://universe.roboflow.com/weeddetection-9usvj/weed_detection-rrlf8/dataset/6

## 1. Business Understanding

### Project Objective
Develop a computer vision model that can:
1. Detect individual weed instances in images
2. Create precise outlines (masks) around each weed
3. Classify each instance by weed type

### Success Criteria
- Instance segmentation with mean average precision (mAP) of at least 0.7
- Classification accuracy of at least 80% for each weed type
- Real-time or near real-time inference (to be deployed on suitable hardware later)

### Use Cases
- Precision agriculture for targeted weed removal
- Agricultural research and weed population monitoring
- Automated farm management systems

## 2. Environment Setup and Data Acquisition

In [ ]:
!nvidia-smi

In [ ]:
!pip install -q accelerate


### 2.1 Setup for Windows 11 with CUDA 12.8

In [ ]:
import os


# Create a base directory for the project on Google Drive (if it doesn't exist)
# Adjust the path '/content/drive/MyDrive/Colab Notebooks/YOLOv8_Weed_Detection' as desired
DRIVE_PROJECT_BASE = '/content/drive/MyDrive/Colab_Notebooks/YOLOv8_Weed_Detection' # <--- CHANGE THIS PATH IF NEEDED
os.makedirs(DRIVE_PROJECT_BASE, exist_ok=True)
print(f"Project base directory on Drive: {DRIVE_PROJECT_BASE}")

print("\nInstalling required libraries...")
# Install YOLOv8, Roboflow, and other common dependencies quietly
!pip install ultralytics roboflow opencv-python matplotlib seaborn scikit-learn pandas pyyaml tqdm -q
# Install albumentations for visualization if needed
!pip install -q albumentations

print("\nDependency installation complete.")

In [ ]:
# Verify CUDA is available and which version is being used
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

In [ ]:
# Import necessary libraries
import os
import sys
import json
import random
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import seaborn as sns
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm

# For YOLOv8
from ultralytics import YOLO

# Set random seed for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Set environment variables for better performance on Windows with CUDA
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
torch.backends.cudnn.benchmark = True  # May improve performance

### 2.2 Download and Explore Dataset

In [ ]:
# Download and extract the dataset from Roboflow
from roboflow import Roboflow

# Initialize Roboflow (You'll need to get your API key from Roboflow)
rf = Roboflow(api_key="HE9CEH5JxJ3U0vXrQTOy")

# Access the project and version
project = rf.workspace("weeddetection-9usvj").project("weed_detection-rrlf8")

# For YOLOv8 segmentation, we need YOLO format
dataset = project.version(6).download("yolov8")

# Get the dataset path
dataset_path = Path(dataset.location)

In [ ]:
# Explore the dataset structure
print(f"Dataset path: {dataset_path}")

# List directories
print("\nDirectories:")
for item in os.listdir(dataset_path):
    if os.path.isdir(os.path.join(dataset_path, item)):
        print(f"- {item}")

# Check for data.yaml file which contains dataset configuration
data_yaml_path = dataset_path / "data.yaml"
if os.path.exists(data_yaml_path):
    print(f"\nData configuration file found: {data_yaml_path}")

    # Display YAML content
    import yaml
    with open(data_yaml_path, 'r') as f:
        data_yaml = yaml.safe_load(f)
        print("\nDataset configuration:")
        for key, value in data_yaml.items():
            print(f"- {key}: {value}")
else:
    print("Data configuration file (data.yaml) not found!")

In [ ]:
# Visualize some examples from the dataset
def visualize_yolo_annotations(dataset_path, num_samples=3):
    # Directories for images and labels
    train_img_dir = dataset_path / "train" / "images"
    train_label_dir = dataset_path / "train" / "labels"

    # Load class names from data.yaml
    with open(dataset_path / "data.yaml", 'r') as f:
        data_cfg = yaml.safe_load(f)
        class_names = data_cfg['names']

    # Get random sample images
    image_files = [f for f in os.listdir(train_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))

    # Create a figure for display
    fig, axes = plt.subplots(1, num_samples, figsize=(15, 5))
    if num_samples == 1:
        axes = [axes]

    # Plot each sample image with annotations
    for i, img_file in enumerate(sample_images):
        # Get corresponding label file
        label_file = os.path.splitext(img_file)[0] + ".txt"
        label_path = train_label_dir / label_file

        # Load image
        img_path = train_img_dir / img_file
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width = image.shape[:2]

        # Load and parse annotation if it exists
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                lines = f.readlines()

            for line in lines:
                # Parse YOLO format annotation
                parts = line.strip().split(' ')
                class_id = int(parts[0])
                # For segmentation masks, format is: class_id x1 y1 x2 y2 x3 y3 ...
                if len(parts) > 5:  # It's a segmentation mask
                    # Convert normalized polygon coordinates to image coordinates
                    polygon = []
                    for j in range(1, len(parts), 2):
                        if j+1 < len(parts):
                            x = float(parts[j]) * width
                            y = float(parts[j+1]) * height
                            polygon.append([x, y])

                    # Draw the polygon
                    polygon = np.array(polygon, dtype=np.int32)
                    color = np.random.randint(0, 255, 3).tolist()
                    cv2.polylines(image, [polygon], True, color, 2)

                    # Get class name
                    class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"

                    # Add label at a position near the polygon
                    if len(polygon) > 0:
                        x, y = polygon[0]
                        cv2.putText(image, class_name, (int(x), int(y-5)),
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
                else:  # It's a bounding box (should not happen for segmentation, but just in case)
                    center_x, center_y, w, h = map(float, parts[1:5])
                    x1 = int((center_x - w/2) * width)
                    y1 = int((center_y - h/2) * height)
                    x2 = int((center_x + w/2) * width)
                    y2 = int((center_y + h/2) * height)

                    color = np.random.randint(0, 255, 3).tolist()
                    cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

                    # Get class name
                    class_name = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
                    cv2.putText(image, class_name, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Display the annotated image
        axes[i].imshow(image)
        axes[i].set_title(f"Image {img_file}")
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

# Visualize sample images
visualize_yolo_annotations(dataset_path, 3)

In [ ]:
import os
from pathlib import Path
import yaml

print(f"Starting detailed label check for dataset at: {dataset_path}")

# --- Configuration ---
try:
    # Load nc (number of classes) from the augmented yaml
    data_yaml_to_check = dataset_path / "data_augmented.yaml"
    if not data_yaml_to_check.exists():
        data_yaml_to_check = dataset_path / "data.yaml" # Fallback to original
        if not data_yaml_to_check.exists():
             raise FileNotFoundError("Cannot find data.yaml or data_augmented.yaml")

    with open(data_yaml_to_check, 'r') as f:
        data_cfg = yaml.safe_load(f)
        if 'nc' not in data_cfg:
             raise ValueError("'nc' (number of classes) not found in YAML.")
        num_classes = int(data_cfg['nc'])
    print(f"Number of classes defined in {data_yaml_to_check.name}: {num_classes}")
except Exception as e:
    print(f"Error loading configuration from YAML: {e}")
    print("Cannot proceed with class ID checks.")
    num_classes = None # Set to None if loading failed

# --- Check Function ---
def check_segmentation_label_file(filepath, num_classes):
    issues = []
    if not os.path.exists(filepath):
        issues.append("File does not exist (should not happen if iterating correctly)")
        return issues
    if os.path.getsize(filepath) == 0:
        issues.append("File is empty (0 bytes)")
        # Don't return yet, check if it has lines anyway (unlikely)

    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()

        if not lines:
             if "File is empty (0 bytes)" not in issues: # Avoid duplicate message
                 issues.append("File exists but contains no lines")
             return issues # No point checking lines if there are none

        has_valid_line = False
        for i, line in enumerate(lines):
            line_num = i + 1
            parts = line.strip().split()
            if not parts: # Skip empty lines within the file
                continue

            # Check number of parts (class_id + at least 2 pairs of coords)
            if len(parts) < 5 or len(parts) % 2 != 1:
                issues.append(f"Line {line_num}: Incorrect number of values ({len(parts)}). Expected class_id + pairs of coords.")
                continue # Skip further checks for this malformed line

            # Check Class ID
            class_id_str = parts[0]
            try:
                class_id = int(class_id_str)
                if num_classes is not None: # Only check if nc was loaded
                     if not (0 <= class_id < num_classes):
                         issues.append(f"Line {line_num}: Class ID '{class_id}' is outside the valid range [0, {num_classes-1}]")
            except ValueError:
                issues.append(f"Line {line_num}: Class ID '{class_id_str}' is not an integer.")
                continue # Skip coordinate checks if class ID is bad

            # Check Coordinates are numeric and in range [0, 1]
            all_coords_valid = True
            for j, coord_str in enumerate(parts[1:]):
                try:
                    coord = float(coord_str)
                    if not (0.0 <= coord <= 1.0):
                         # Allow slightly outside range due to augmentation/rounding, but flag significant issues
                         if not (-0.1 <= coord <= 1.1):
                             issues.append(f"Line {line_num}: Coordinate {j+1} ('{coord}') is significantly outside the expected range [0, 1]")
                             all_coords_valid = False
                         # else: # Optionally add warning for minor deviations
                         #    issues.append(f"Warning - Line {line_num}: Coordinate {j+1} ('{coord}') is slightly outside [0, 1]")

                except ValueError:
                    issues.append(f"Line {line_num}: Coordinate {j+1} ('{coord_str}') is not a number.")
                    all_coords_valid = False

            if all_coords_valid:
                 has_valid_line = True # Found at least one line that looks okay

        # After checking all lines
        if not has_valid_line and lines and not any("Incorrect number of values" in iss or "Class ID" in iss for iss in issues):
             # File had lines, but none were considered valid segmentation lines after checks
             issues.append("File contains lines, but none seem to be valid segmentation annotations.")
        elif not has_valid_line and not lines:
             if "File exists but contains no lines" not in issues and "File is empty (0 bytes)" not in issues:
                  issues.append("File appears empty or contains only invalid lines.")


    except Exception as e:
        issues.append(f"Failed to read or process file: {str(e)}")

    return issues

# --- Iterate and Check ---
total_issues = 0
max_files_to_print = 20 # Limit printing excessively long lists
problem_files = {}

for split in ['train', 'valid', 'test']:
    label_dir = dataset_path / split / "labels"
    if not label_dir.exists():
        print(f"Directory not found: {label_dir} (Skipping)")
        continue

    print(f"\nChecking labels in: {label_dir}...")
    found_files = list(label_dir.glob("*.txt"))
    if not found_files:
         print(f"No .txt label files found in {label_dir}.")
         continue

    print(f"Found {len(found_files)} .txt files. Checking content...")
    split_issue_count = 0
    for label_file in tqdm(found_files, desc=f"Checking {split} labels"):
        issues = check_segmentation_label_file(label_file, num_classes)
        if issues:
            problem_files[str(label_file)] = issues
            total_issues += 1
            split_issue_count += 1

    print(f"Finished checking {split} split. Found issues in {split_issue_count} files.")


# --- Report Results ---
print(f"\n--- Detailed Label Check Summary ---")
print(f"Total files with potential issues found: {total_issues}")

if problem_files:
    print(f"\nFiles with issues (showing up to {max_files_to_print}):")
    count = 0
    for filepath, issues in problem_files.items():
        if count >= max_files_to_print:
            print(f"... and {total_issues - count} more files with issues.")
            break
        print(f"\nFile: {filepath}")
        for issue in issues:
            print(f"  - {issue}")
        count += 1
else:
    print("\nNo potential issues detected in label files based on these checks.")

print("--- End of Summary ---")


In [ ]:
# Analyze dataset statistics
def analyze_dataset(dataset_path):
    # Load class names from data.yaml
    with open(dataset_path / "data.yaml", 'r') as f:
        data_cfg = yaml.safe_load(f)
        class_names = data_cfg['names']

    # Initialize counters
    stats = {
        'train': {'images': 0, 'instances_per_class': {cls: 0 for cls in range(len(class_names))}},
        'valid': {'images': 0, 'instances_per_class': {cls: 0 for cls in range(len(class_names))}},
        'test': {'images': 0, 'instances_per_class': {cls: 0 for cls in range(len(class_names))}}
    }

    # Go through each split
    for split in ['train', 'valid', 'test']:
        label_dir = dataset_path / split / "labels"
        if not os.path.exists(label_dir):
            continue

        # Count images
        stats[split]['images'] = len([f for f in os.listdir(label_dir) if f.endswith('.txt')])

        # Count instances per class
        for label_file in os.listdir(label_dir):
            if not label_file.endswith('.txt'):
                continue

            with open(label_dir / label_file, 'r') as f:
                for line in f:
                    parts = line.strip().split(' ')
                    if len(parts) >= 1:
                        class_id = int(parts[0])
                        if class_id in stats[split]['instances_per_class']:
                            stats[split]['instances_per_class'][class_id] += 1

    # Print statistics
    print("Dataset Statistics:")
    for split, data in stats.items():
        print(f"\n{split.capitalize()} set:")
        print(f"  - Number of images: {data['images']}")
        print("  - Instances per class:")
        for class_id, count in data['instances_per_class'].items():
            if count > 0:  # Only show classes that have instances
                print(f"    - {class_names[class_id]}: {count}")

    # Visualize class distribution
    plt.figure(figsize=(12, 6))

    # Prepare data for plotting
    df_data = []
    for split, data in stats.items():
        for class_id, count in data['instances_per_class'].items():
            if count > 0:  # Only include classes with instances
                df_data.append({
                    'Split': split.capitalize(),
                    'Class': class_names[class_id],
                    'Count': count
                })

    df = pd.DataFrame(df_data)

    # Create grouped bar chart
    sns.barplot(x='Class', y='Count', hue='Split', data=df)
    plt.title('Class Distribution Across Dataset Splits')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

    # Return statistics for further use
    return stats

# Analyze the dataset
dataset_stats = analyze_dataset(dataset_path)

### 2.3 Data Cleaning, Preprocessing, and Augmentation

Before training, we will:
1. Check for and handle corrupted images
2. Analyze class imbalance and handle it
3. Set up custom data augmentation strategies for weed detection
4. Create a custom YAML file with the optimized configuration

In [ ]:
# Function to check for corrupted images
def check_corrupt_images(dataset_path):
    print("Checking for corrupted images...")
    corrupted_count = 0

    for split in ['train', 'valid', 'test']:
        img_dir = dataset_path / split / "images"
        if not os.path.exists(img_dir):
            continue

        print(f"Checking {split} set...")

        for img_file in tqdm(os.listdir(img_dir), desc=f"Checking {split} images"):
            if not img_file.endswith(('.jpg', '.jpeg', '.png')):
                continue

            img_path = img_dir / img_file
            try:
                # Try to open and process the image
                img = cv2.imread(str(img_path))
                if img is None or img.size == 0:
                    print(f"Corrupted image found: {img_path}")
                    corrupted_count += 1
                    continue

                # Check if image is grayscale and convert to RGB if needed
                if len(img.shape) == 2 or img.shape[2] == 1:
                    print(f"Converting grayscale image to RGB: {img_path}")
                    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
                    cv2.imwrite(str(img_path), img)

                # Check for unusual aspect ratios
                height, width = img.shape[:2]
                aspect_ratio = width / height
                if aspect_ratio > 3 or aspect_ratio < 0.33:
                    print(f"Unusual aspect ratio ({aspect_ratio:.2f}) detected: {img_path}")

            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")
                corrupted_count += 1

    print(f"Finished checking. Found {corrupted_count} corrupted images.")
    return corrupted_count

# Check for corrupted images
corrupted_count = check_corrupt_images(dataset_path)

In [ ]:
# Function to check for inconsistencies between images and labels
def check_label_consistency(dataset_path):
    print("Checking for inconsistencies between images and labels...")
    issue_count = 0

    for split in ['train', 'valid', 'test']:
        img_dir = dataset_path / split / "images"
        label_dir = dataset_path / split / "labels"

        if not os.path.exists(img_dir) or not os.path.exists(label_dir):
            continue

        print(f"Checking {split} set...")

        # Check images without labels
        images = {os.path.splitext(f)[0] for f in os.listdir(img_dir)
                 if f.endswith(('.jpg', '.jpeg', '.png'))}
        labels = {os.path.splitext(f)[0] for f in os.listdir(label_dir)
                 if f.endswith('.txt')}

        img_no_label = images - labels
        if img_no_label:
            print(f"Found {len(img_no_label)} images without labels in {split} set")
            for i, img_name in enumerate(list(img_no_label)[:5]):
                print(f"  {i+1}. {img_name}")
            if len(img_no_label) > 5:
                print(f"  ... and {len(img_no_label) - 5} more")
            issue_count += len(img_no_label)

        # Check labels without images
        label_no_img = labels - images
        if label_no_img:
            print(f"Found {len(label_no_img)} labels without images in {split} set")
            for i, label_name in enumerate(list(label_no_img)[:5]):
                print(f"  {i+1}. {label_name}")
            if len(label_no_img) > 5:
                print(f"  ... and {len(label_no_img) - 5} more")
            issue_count += len(label_no_img)

        # Check for empty label files
        empty_labels = 0
        for label_file in labels:
            label_path = label_dir / f"{label_file}.txt"
            if os.path.exists(label_path) and os.path.getsize(label_path) == 0:
                empty_labels += 1

        if empty_labels > 0:
            print(f"Found {empty_labels} empty label files in {split} set")
            issue_count += empty_labels

    print(f"Finished checking. Found {issue_count} inconsistencies.")
    return issue_count

# Check for label inconsistencies
issue_count = check_label_consistency(dataset_path)

In [ ]:
import os
from pathlib import Path
from tqdm.auto import tqdm # Use tqdm for progress bars

print(f"Searching for empty label files within dataset at: {dataset_path}")

# --- Function to find empty label files ---
def find_empty_labels(base_dataset_path):
    empty_files = []
    for split in ['train', 'valid', 'test']:
        label_dir = Path(base_dataset_path) / split / "labels"
        if not label_dir.exists():
            print(f"Warning: Label directory not found for {split} split: {label_dir}")
            continue

        for label_file in label_dir.glob("*.txt"):
            try:
                # Check if file size is 0 bytes
                if label_file.stat().st_size == 0:
                    empty_files.append(label_file)
            except Exception as e:
                print(f"Warning: Could not check size of {label_file}: {e}")
    return empty_files

# --- Find the empty files ---
empty_label_paths = find_empty_labels(dataset_path)

if not empty_label_paths:
    print("No empty label files found. No removal needed.")
else:
    print(f"Found {len(empty_label_paths)} empty label files. Proceeding with removal...")
    print("Empty files found:")
    for p in empty_label_paths:
        print(f"- {p}")

    # --- Function to remove labels and corresponding images ---
    def remove_empty_labels_and_images(label_paths_to_remove):
        removed_labels_count = 0
        removed_images_count = 0
        label_errors = []
        image_errors = []

        print("\nStarting removal process...")
        for label_path in tqdm(label_paths_to_remove, desc="Removing files"):
            # --- Remove Label File ---
            if label_path.exists():
                try:
                    os.remove(label_path)
                    # print(f"Removed empty label file: {label_path}") # Optional: less verbose
                    removed_labels_count += 1
                except Exception as e:
                    error_msg = f"Failed to remove label {label_path}: {e}"
                    print(error_msg)
                    label_errors.append(error_msg)
                    continue # Skip image removal if label removal failed
            else:
                # This case shouldn't happen if find_empty_labels worked correctly
                print(f"Label file not found (already removed?): {label_path}")
                continue

            # --- Find and Remove Corresponding Image File ---
            label_filename_base = label_path.stem # Get filename without .txt
            # Construct path relative to dataset_path structure
            image_dir = label_path.parent.parent / "images"

            image_found_and_removed = False
            for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']: # Check common extensions (case-insensitive)
                image_path = image_dir / f"{label_filename_base}{ext}"
                if image_path.exists():
                    try:
                        os.remove(image_path)
                        # print(f"-> Removed corresponding image file: {image_path}") # Optional: less verbose
                        removed_images_count += 1
                        image_found_and_removed = True
                        break # Stop looking once found and removed
                    except Exception as e:
                         error_msg = f"-> Failed to remove image {image_path}: {e}"
                         print(error_msg)
                         image_errors.append(error_msg)
                         break # Stop trying for this image if error occurs

            if not image_found_and_removed:
                 warn_msg = f"-> Warning: Could not find corresponding image for {label_path.name} in {image_dir}"
                 print(warn_msg)
                 image_errors.append(warn_msg)

        # --- Print Summary ---
        print(f"\n--- Removal Summary ---")
        print(f"Attempted removal for {len(label_paths_to_remove)} empty label files.")
        print(f"Successfully removed labels: {removed_labels_count}")
        print(f"Successfully removed corresponding images: {removed_images_count}")
        if label_errors:
            print(f"\nErrors removing labels ({len(label_errors)}):")
            for err in label_errors[:5]: print(f"  - {err}") # Print first 5 errors
            if len(label_errors) > 5: print("  ...")
        if image_errors:
            print(f"\nErrors/Warnings finding/removing images ({len(image_errors)}):")
            for err in image_errors[:5]: print(f"  - {err}") # Print first 5 errors/warnings
            if len(image_errors) > 5: print("  ...")
        print("--- End of Summary ---")

    # --- Execute the removal ---
    remove_empty_labels_and_images(empty_label_paths)


In [ ]:
# Function to handle class imbalance
def handle_class_imbalance(dataset_path, dataset_stats):
    print("Analyzing class imbalance...")

    # Load class names from data.yaml
    with open(dataset_path / "data.yaml", 'r') as f:
        data_cfg = yaml.safe_load(f)
        class_names = data_cfg['names']

    # Get class distribution in training set
    train_instances = dataset_stats['train']['instances_per_class']

    # Sort classes by number of instances
    sorted_classes = sorted(train_instances.items(), key=lambda x: x[1])

    # Identify minority and majority classes
    minority_classes = []
    majority_classes = []

    # Get the mean number of instances per class
    mean_instances = sum(train_instances.values()) / len(train_instances)
    print(f"Mean instances per class: {mean_instances:.2f}")

    for class_id, count in sorted_classes:
        class_name = class_names[class_id]
        if count < 0.5 * mean_instances:
            minority_classes.append((class_id, class_name, count))
        elif count > 1.5 * mean_instances:
            majority_classes.append((class_id, class_name, count))

    print("\nMinority classes (below 50% of mean):")
    for class_id, class_name, count in minority_classes:
        print(f"  - {class_name}: {count} instances ({count/mean_instances:.2f}x mean)")

    print("\nMajority classes (above 150% of mean):")
    for class_id, class_name, count in majority_classes:
        print(f"  - {class_name}: {count} instances ({count/mean_instances:.2f}x mean)")

    # Calculate class weights for loss function
    max_instances = max(train_instances.values())
    class_weights = {class_id: max_instances / max(1, count) for class_id, count in train_instances.items()}

    print("\nCalculated class weights for loss function:")
    for class_id, weight in class_weights.items():
        print(f"  - {class_names[class_id]}: {weight:.2f}")

    # Return class weights dictionary for use during training
    return {
        'minority_classes': minority_classes,
        'majority_classes': majority_classes,
        'class_weights': class_weights
    }

# Handle class imbalance
imbalance_info = handle_class_imbalance(dataset_path, dataset_stats)

In [ ]:
# Define custom data augmentation strategies for weed segmentation
def create_augmentation_config(dataset_path, imbalance_info):
    print("Creating custom data augmentation configuration...")

    # Load the original YAML config
    data_yaml_path = dataset_path / "data.yaml"
    with open(data_yaml_path, 'r') as f:
        data_cfg = yaml.safe_load(f)

    # Create a custom YAML configuration with augmentations
    augmented_cfg = data_cfg.copy()

    # Define stronger augmentation for minority classes
    minority_class_ids = [c[0] for c in imbalance_info['minority_classes']]

    # Advanced augmentation config for YOLOv8
    augmented_cfg['augment'] = True

    # YOLOv8 hyperparameters with augmentation settings
    augmented_cfg['hyp'] = {
        # Augmentation probabilities and parameters
        'mosaic': 1.0,           # Mosaic augmentation probability
        'mixup': 0.2,            # Mixup augmentation probability
        'copy_paste': 0.3,       # Copy-paste augmentation probability (good for instances)
        'degrees': 20.0,         # Rotation augmentation degrees
        'translate': 0.2,        # Translation augmentation magnitude
        'scale': 0.2,            # Scale augmentation magnitude
        'shear': 0.2,            # Shear augmentation magnitude
        'perspective': 0.0,      # Perspective augmentation magnitude (reduce for close-up weed images)
        'flipud': 0.1,           # Vertical flip probability
        'fliplr': 0.5,           # Horizontal flip probability
        'hsv_h': 0.015,          # HSV hue augmentation magnitude
        'hsv_s': 0.2,            # HSV saturation augmentation magnitude
        'hsv_v': 0.2,            # HSV value augmentation magnitude (brightness)

        # Training hyperparameters (will be overridden by train function params)
        'lr0': 0.001,
        'weight_decay': 0.0005,
        'warmup_epochs': 3.0,
        'warmup_momentum': 0.8,
        'warmup_bias_lr': 0.1,
        'box': 7.5,              # Box loss weight
        'cls': 0.5,              # Class loss weight
        'dfl': 1.5,              # Distribution focal loss weight
        'masks': 1.0,            # Mask loss weight (important for segmentation)
    }

    # Set class weights to handle class imbalance
    class_weights = imbalance_info['class_weights']
    # Normalize weights to be centered around 1
    avg_weight = sum(class_weights.values()) / len(class_weights)
    normalized_weights = {str(k): v / avg_weight for k, v in class_weights.items()}

    augmented_cfg['class_weights'] = normalized_weights

    # Save the augmented config
    augmented_yaml_path = dataset_path / "data_augmented.yaml"
    with open(augmented_yaml_path, 'w') as f:
        yaml.dump(augmented_cfg, f, default_flow_style=False)

    print(f"Created augmented configuration at: {augmented_yaml_path}")
    return augmented_yaml_path

# Create augmentation configuration
augmented_data_yaml = create_augmentation_config(dataset_path, imbalance_info)

In [ ]:
# Function to visualize the effects of data augmentation
def visualize_augmentation(dataset_path, num_samples=3):
    print("Visualizing data augmentation effects...")

    # Get a few sample images from training set
    train_img_dir = dataset_path / "train" / "images"
    if not os.path.exists(train_img_dir):
        print("Training image directory not found!")
        return

    image_files = [f for f in os.listdir(train_img_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
    if len(image_files) == 0:
        print("No training images found!")
        return

    # Randomly sample images
    sample_images = random.sample(image_files, min(num_samples, len(image_files)))

    # Initialize transforms
    import albumentations as A
    from albumentations.pytorch import ToTensorV2

    # Define standard augmentation pipeline
    standard_augs = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1, p=0.8),
    ])

    # Define stronger augmentation pipeline
    strong_augs = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.RandomRotate90(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20, p=0.7),
        A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.8),
        A.Blur(blur_limit=3, p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        A.GaussNoise(var_limit=(10, 50), p=0.3),
    ])

    # Visualize each sample with different augmentations
    for img_file in sample_images:
        img_path = train_img_dir / img_file

        # Read image
        img = cv2.imread(str(img_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Create augmented versions
        aug_standard = standard_augs(image=img)['image']
        aug_strong = strong_augs(image=img)['image']

        # Display original and augmented
        fig, axes = plt.subplots(1, 3, figsize=(18, 6))

        axes[0].imshow(img)
        axes[0].set_title("Original Image")
        axes[0].axis('off')

        axes[1].imshow(aug_standard)
        axes[1].set_title("Standard Augmentation")
        axes[1].axis('off')

        axes[2].imshow(aug_strong)
        axes[2].set_title("Strong Augmentation")
        axes[2].axis('off')

        plt.suptitle(f"Augmentation Effects on {img_file}", fontsize=16)
        plt.tight_layout()
        plt.show()

# Install albumentations for visualization
!pip install -q albumentations

# Visualize augmentation effects
visualize_augmentation(dataset_path, num_samples=2)

In [ ]:
# Function to create a preprocessing summary
def create_preprocessing_summary(dataset_path, corrupted_count, issue_count, imbalance_info):
    print("\n=============================================")
    print("DATA PREPROCESSING SUMMARY")
    print("=============================================\n")

    # Dataset overview
    print("Dataset Location:", dataset_path)

    # Load class names from data.yaml
    # Assumes yaml was imported globally earlier
    # import yaml
    with open(dataset_path / "data.yaml", 'r') as f:
        data_cfg = yaml.safe_load(f)
        class_names = data_cfg['names']

    print(f"Number of Classes: {len(class_names)}")
    print(f"Classes: {', '.join(class_names)}")

    # Data cleaning results
    print("\nData Cleaning Results:")
    print(f"  - Corrupted Images Checked: {corrupted_count} found (Note: Function attempts conversion/prints warnings)")
    print(f"  - Label Inconsistencies Checked: {issue_count} found (Note: Function reports counts)")

    # Class imbalance summary
    print("\nClass Imbalance Summary:")
    if imbalance_info['minority_classes']:
        print(f"  - Minority Classes (below 50% mean instances): {len(imbalance_info['minority_classes'])}")
        for _, name, count in imbalance_info['minority_classes']:
            print(f"    * {name}: {count} instances")
    else:
        print("  - No significant minority classes identified based on threshold.")

    if imbalance_info['majority_classes']:
         print(f"  - Majority Classes (above 150% mean instances): {len(imbalance_info['majority_classes'])}")
         for _, name, count in imbalance_info['majority_classes']:
             print(f"    * {name}: {count} instances")
    else:
         print("  - No significant majority classes identified based on threshold.")


    # Augmentation strategy (as configured in YAML)
    print("\nAugmentation Strategy (Enabled in data_augmented.yaml):")
    print("  - Augmentations like mosaic, mixup, copy-paste, flips, color jitter, etc., are probabilistically applied during training based on hyperparameters.")
    print("  - While helpful for imbalance, these augmentations are generally applied across batches, not exclusively targeted to minority class instances within the standard training loop.")
    print("  - Class weights were calculated and included in the configuration to directly address imbalance during loss calculation.")

    # Final dataset configuration
    print("\nFinal Dataset Configuration:")
    print(f"  - Augmented config file used for training: data_augmented.yaml")
    print(f"  - Class weights applied via config: {bool(imbalance_info['class_weights'])}")

    print("\nPreprocessing checks completed. Ready for model training.")

# Create preprocessing summary
# Make sure the input variables (dataset_path, corrupted_count, issue_count, imbalance_info) exist
try:
    # Assumes yaml package was imported in a previous cell
    create_preprocessing_summary(dataset_path, corrupted_count, issue_count, imbalance_info)
except NameError as e:
    # This error message is intentional if prerequisite cells haven't been run
    print(f"Error creating summary: Make sure all required variables are defined. Missing variable: {e.name}")
except ImportError:
    print("Error: The 'yaml' package needs to be imported. Please ensure PyYAML is installed and imported in a previous cell.")


### 2.3.3 Validate Polygon Geometry and Remove Invalid Labels/Images

Even if label files exist and are non-empty, they might contain invalid polygon definitions (e.g., self-intersecting, too few points, collapsed area) which can cause errors during segmentation training. This step uses the `shapely` library to validate polygon geometry.

**Action:**
- Scans all `.txt` label files in `train`, `valid`, and `test` splits.
- Identifies files containing one or more invalid or near-zero area polygons.
- **Automatically DELETES** these problematic label files **AND** their corresponding image files.

**⚠️ Warning:** This step modifies your dataset by deleting files. Ensure you have a backup or can easily re-download the dataset if needed.

In [ ]:
# === Step 2.3.3: Validate Polygon Geometry & Remove Invalid Data ===

# --- Ensure Shapely is installed ---
try:
    from shapely.geometry import Polygon
    from shapely.errors import GEOSException
    print("Shapely is already installed.")
except ImportError:
    print("Installing Shapely for polygon validation...")
    !pip install shapely -q
    from shapely.geometry import Polygon
    from shapely.errors import GEOSException
    print("Shapely installed.")

import os
import yaml
from pathlib import Path
from tqdm.auto import tqdm

print(f"\nStarting polygon validation & removal for dataset: {dataset_path}")
print("WARNING: This will DELETE label files with invalid polygons AND their corresponding images.")

# --- Configuration ---
num_classes = None # Initialize
try:
    # Load nc from the most relevant YAML (augmented if exists, else original)
    data_yaml_to_check = dataset_path / "data_augmented.yaml"
    if not data_yaml_to_check.exists():
        data_yaml_to_check = dataset_path / "data.yaml"

    if data_yaml_to_check.exists():
         with open(data_yaml_to_check, 'r') as f:
             data_cfg = yaml.safe_load(f)
             if 'nc' in data_cfg:
                  num_classes = int(data_cfg['nc'])
                  print(f"Number of classes from {data_yaml_to_check.name}: {num_classes}")
             else: print("Warning: 'nc' not found in YAML.")
    else: print(f"Warning: YAML file not found at {data_yaml_to_check}")
except Exception as e: print(f"Warning: Error loading YAML: {e}")

# --- Validation Function (returns True if issues found) ---
def check_file_for_geometry_issues(filepath, num_classes):
    """Checks a label file for empty status or invalid polygon geometry."""
    if not filepath.exists() or filepath.stat().st_size == 0: return True # Empty or missing

    try:
        with open(filepath, 'r') as f: lines = f.readlines()
        if not lines: return True # No lines

        for i, line in enumerate(lines):
            parts = line.strip().split()
            if not parts: continue

            if len(parts) < 7 or len(parts) % 2 != 1: return True # Basic format issue

            # Validate Class ID
            try:
                class_id = int(parts[0])
                if num_classes is not None and not (0 <= class_id < num_classes): return True
            except ValueError: return True # Class ID not integer

            # Validate Coordinates & Geometry
            coords = []
            for j in range(1, len(parts), 2):
                try:
                    x = float(parts[j]); y = float(parts[j+1])
                    coords.append((x, y))
                except (ValueError, IndexError): return True # Coord not number or pair missing

            if len(coords) < 3: return True # Not enough points

            try:
                polygon = Polygon(coords)
                area_threshold = 1e-7 # Threshold for collapsed polygon
                if not polygon.is_valid or polygon.area < area_threshold: return True # Invalid geometry or area
            except (GEOSException, ValueError, Exception): return True # Error creating polygon

    except Exception: return True # Issue reading file
    return False # No issues found

# --- Find Files with Issues ---
files_to_remove = []
print("\nScanning files for geometry issues...")
for split in ['train', 'valid', 'test']:
    label_dir = dataset_path / split / "labels"
    if not label_dir.exists(): continue

    print(f"Scanning {split} labels...")
    current_split_files = list(label_dir.glob("*.txt"))
    if not current_split_files: continue

    for label_file in tqdm(current_split_files, desc=f"Scanning {split}"):
        if check_file_for_geometry_issues(label_file, num_classes):
            files_to_remove.append(label_file)

print(f"\nScan complete. Found {len(files_to_remove)} label files flagged for removal (empty or invalid geometry).")

# --- Automated Removal ---
if files_to_remove:
    print(f"\nProceeding to remove {len(files_to_remove)} label files and their corresponding images...")
    removed_labels_count = 0
    removed_images_count = 0
    errors = []

    for label_path in tqdm(files_to_remove, desc="Removing files"):
        label_removed = False
        # Remove Label
        try:
            if label_path.exists():
                os.remove(label_path)
                removed_labels_count += 1
                label_removed = True
        except Exception as e: errors.append(f"Label {label_path}: {e}")

        # Find and Remove Corresponding Image
        if label_removed:
            label_filename_base = label_path.stem
            image_dir = label_path.parent.parent / "images"
            image_found = False
            for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
                image_path = image_dir / f"{label_filename_base}{ext}"
                if image_path.exists():
                    image_found = True
                    try:
                        os.remove(image_path)
                        removed_images_count += 1
                        break
                    except Exception as e:
                        errors.append(f"Image {image_path}: {e}")
                        break
            # Optional: Add warning if image not found, but can be noisy
            # if not image_found: errors.append(f"Warning: Image not found for {label_path.name}")

    # --- Removal Summary ---
    print(f"\n--- Removal Summary ---")
    print(f"Labels removed: {removed_labels_count}/{len(files_to_remove)}")
    print(f"Images removed: {removed_images_count}")
    if errors:
        print(f"Encountered {len(errors)} errors/warnings (showing first 5):")
        for i, err in enumerate(errors[:5]): print(f"  - {err}")
else:
    print("No files needed removal based on geometry checks.")

print("\n--- Polygon Validation and Automated Removal Process Complete ---")

# Optional: Re-run the dataset analysis cell to see updated counts
# print("\nRe-running dataset analysis after cleaning...")
# try:
#     dataset_stats = analyze_dataset(dataset_path)
# except NameError:
#     print("analyze_dataset function not defined or dataset_path missing.")


## 3. Modeling with YOLOv8

### 3.1 Model Selection and Configuration

In [ ]:
# Check YOLOv8 available models
print("Available YOLOv8 segmentation models:")
print("- yolov8n-seg.pt (smallest, fastest)")
print("- yolov8s-seg.pt (small)")
print("- yolov8m-seg.pt (medium)")
print("- yolov8l-seg.pt (large)")
print("- yolov8x-seg.pt (extra large, most accurate)")

# Select model based on your system's capabilities
# Select the large model for a balance of speed and accuracy
MODEL_NAME = "yolov8l-seg.pt"

# Load a pre-trained YOLOv8 segmentation model
model = YOLO(MODEL_NAME)
print(f"\nLoaded {MODEL_NAME} model")

In [ ]:
# Create a project directory for output
project_dir = Path("./weed_detection_yolov8")
os.makedirs(project_dir, exist_ok=True)
print(f"Project directory: {project_dir}")

# Use our augmented data.yaml instead of the original one
data_yaml_path = augmented_data_yaml
print(f"Using augmented configuration: {data_yaml_path}")

### 3.2 Model Training

In [ ]:
import torch # Ensure torch is imported
# Assuming model, data_yaml_path, project_dir are defined earlier in your notebook

print("Attempting training...")

# Determine device setting based on GPU availability
if torch.cuda.is_available() and torch.cuda.device_count() >= 2:
    # Attempt to use the first two GPUs if available
    device_setting = [0, 1]
    print(f"Configured to use GPUs: {device_setting}")
    # Note: Effective batch size will be batch * num_gpus.
    # You might want to increase the 'batch' parameter below (e.g., to 16)
    # if memory allows, to utilize both GPUs efficiently.
    batch_size = 8 # Keep original or increase (e.g., 16)
else:
    # Fallback to single GPU or CPU if 2 GPUs aren't available
    device_setting = 0 if torch.cuda.is_available() else 'cpu'
    print(f"Less than 2 GPUs detected. Using device: {device_setting}")
    batch_size = 8 # Keep original batch size

results = model.train(
    data=str(data_yaml_path),  # Path to your data_augmented.yaml
    epochs=30,                  # Run only a few epochs for testing
    imgsz=640,
    batch=batch_size,           # Using the determined batch size
    patience=10,                # Lower patience for quick test
    device=device_setting,      # Use the list [0, 1] or fallback device
    project=str(project_dir),  # Save to Drive output folder
    name="weed_segmentation_debug_multi_gpu", # Updated run name
    pretrained=True,
    optimizer="Adam",
    lr0=0.001,
    single_cls=False,
    dropout=0.0,
    augment=True,
    cache=False,               # Consider setting cache='ram' if you have enough RAM
    verbose=True,
    amp=True,                  # Keep Automatic Mixed Precision
    close_mosaic=0,            # Ensure mosaic is off if augment=False doesn't handle it
    cos_lr=True,
    overlap_mask=True,
    mask_ratio=4,
    task='segment',
    workers=4                  # Added workers for potentially faster data loading
)
print("Training function called.") # Changed message slightly as completion isn't guaranteed if it hangs


### 3.3 Model Evaluation

In [ ]:
# Path to best model weights from training
import os
from pathlib import Path
import yaml
from ultralytics import YOLO

# Ensure project_dir is defined correctly (should match the one used in model.train)
# Example for Colab: project_dir = Path(DRIVE_PROJECT_BASE) / "training_outputs"
# Example for local: project_dir = Path("./weed_detection_yolov8") # Or wherever you saved it
if 'project_dir' not in locals() or not project_dir.exists():
    # Attempt to redefine if missing, adjust this path if necessary
    project_dir = Path("./weed_detection_yolov8")
    if not project_dir.exists():
        raise FileNotFoundError(f"Project directory '{project_dir}' not found. Please ensure training ran and the path is correct.")
    else:
        print(f"Project directory redefined to: {project_dir}")

print(f"Looking for training runs inside: {project_dir}")

# --- Find Available Training Runs ---
available_runs = sorted([d for d in project_dir.iterdir() if d.is_dir() and (d / 'weights').exists()], key=os.path.getmtime, reverse=True)

if not available_runs:
    raise FileNotFoundError(f"No training run directories with a 'weights' subfolder found in {project_dir}. Did the training complete successfully?")

print("\nAvailable training runs (most recent first):")
for i, run_path in enumerate(available_runs):
    print(f"  {i}: {run_path.name}")

# --- Select the Run to Evaluate ---
# Option 1: Automatically select the most recent run
selected_run_path = available_runs[0]
print(f"\nAutomatically selected the most recent run: {selected_run_path.name}")
# Option 2: Manually specify the run name (uncomment and edit if needed)
# correct_run_name = "weed_segmentation_debug_multi_gpu" # <--- UPDATE THIS MANUALLY IF NEEDED
# selected_run_path = project_dir / correct_run_name
# if not selected_run_path.exists() or not (selected_run_path / 'weights').exists():
#     raise FileNotFoundError(f"Specified run directory '{selected_run_path}' or its 'weights' subfolder not found.")
# print(f"\nManually selected run: {selected_run_path.name}")
# -------------------------------------------------

train_output_dir = selected_run_path
print(f"\nAttempting to load results from selected run: {train_output_dir}")

# Find best.pt or fallback to last.pt within the selected run
weights_dir = train_output_dir / "weights"
print(f"Looking for weights in: {weights_dir}")

# This check should now pass if available_runs logic worked
if not weights_dir.exists():
     raise FileNotFoundError(f"Weights directory not found: {weights_dir}. This should not happen if run selection worked.")

best_model_path = weights_dir / "best.pt"
last_model_path = weights_dir / "last.pt"

if best_model_path.exists():
    initial_model_to_load = best_model_path # Store path for fine-tuning
    print(f"Found best model: {initial_model_to_load}")
elif last_model_path.exists():
    initial_model_to_load = last_model_path
    print(f"WARNING: best.pt not found in {weights_dir}. Using last model: {initial_model_to_load}")
else:
    raise FileNotFoundError(f"Could not find 'best.pt' or 'last.pt' in {weights_dir}")

print(f"Loading initial model from: {initial_model_to_load}")

# Load the trained model
initial_trained_model = YOLO(initial_model_to_load)

# Validate on validation set - Ensure task is segment here too for consistency
print(f"Validating initial model using data: {data_yaml_path}")
# Make sure data_yaml_path points to the augmented yaml if that was used for training
# Make sure yaml is imported if needed for metric display later
import yaml
initial_val_results = initial_trained_model.val(data=str(data_yaml_path), task='segment', imgsz=640, batch=8) # Use appropriate batch size for validation

# Display validation metrics for segmentation and STORE them
print("\nInitial Validation Metrics (Segmentation):")
initial_metrics_summary = {}

# --- Access Segmentation Metrics ---
metrics = None
if hasattr(initial_val_results, 'seg'):
    metrics = initial_val_results.seg
    print("Accessed metrics via initial_val_results.seg")
elif hasattr(initial_val_results, 'mask'): # Fallback check
     metrics = initial_val_results.mask
     print("Accessed metrics via initial_val_results.mask")

# --- Print and Store Overall Metrics ---
if metrics and hasattr(metrics, 'map') and hasattr(metrics, 'map50') and hasattr(metrics, 'map75'):
    initial_metrics_summary['Mask mAP50-95'] = metrics.map
    initial_metrics_summary['Mask mAP50'] = metrics.map50
    initial_metrics_summary['Mask mAP75'] = metrics.map75
    print(f"Mask mAP50-95: {metrics.map:.4f}")
    print(f"Mask mAP50: {metrics.map50:.4f}")
    print(f"Mask mAP75: {metrics.map75:.4f}") # Note: mAP75 might not always be populated depending on settings/version

    # --- Print and Store Box Precision/Recall ---
    box_metrics = None
    if hasattr(initial_val_results, 'box'):
        box_metrics = initial_val_results.box

    # Check if precision/recall attributes exist and are sequences (like lists/arrays) with at least one element
    if box_metrics and hasattr(box_metrics, 'p') and hasattr(box_metrics, 'r') and \
       hasattr(box_metrics.p, '__len__') and len(box_metrics.p) > 0 and \
       hasattr(box_metrics.r, '__len__') and len(box_metrics.r) > 0:
         initial_metrics_summary['Box Precision'] = box_metrics.p[0]
         initial_metrics_summary['Box Recall'] = box_metrics.r[0]
         print(f"Box Precision: {box_metrics.p[0]:.4f}") # Access first element
         print(f"Box Recall: {box_metrics.r[0]:.4f}") # Access first element
    else:
         # Sometimes P/R might be single values directly
         if box_metrics and hasattr(box_metrics, 'p') and hasattr(box_metrics, 'r'):
              try:
                   initial_metrics_summary['Box Precision'] = float(box_metrics.p)
                   initial_metrics_summary['Box Recall'] = float(box_metrics.r)
                   print(f"Box Precision: {float(box_metrics.p):.4f}")
                   print(f"Box Recall: {float(box_metrics.r):.4f}")
              except TypeError:
                    print("Box precision/recall metrics found but could not be converted to float.")
         else:
              print("Box precision/recall metrics not found or structure differs.")

    # --- Display Per-Class Mask Metrics ---
    # Check for the necessary attributes in the segmentation metrics object
    if hasattr(metrics, 'ap_class_index') and hasattr(metrics, 'ap50') and \
       metrics.ap_class_index is not None and metrics.ap50 is not None and \
       hasattr(metrics.ap_class_index, '__len__') and len(metrics.ap_class_index) > 0 and \
       hasattr(metrics.ap50, '__len__') and len(metrics.ap50) == len(metrics.ap_class_index):

        class_names = []
        try:
            # Ensure yaml is imported (might have been done earlier)
            import yaml
            # Use the same YAML path variable used for training/validation
            # Ensure data_yaml_path is defined correctly before this cell
            with open(data_yaml_path, 'r') as f:
                data_config = yaml.safe_load(f)
                if 'names' in data_config and isinstance(data_config['names'], list):
                    class_names = data_config['names']
                else:
                    print(f"Warning: 'names' key not found or not a list in {data_yaml_path}")
        except FileNotFoundError:
            print(f"Error: Data YAML file not found at {data_yaml_path}")
        except ImportError:
             print("Error: PyYAML package is required but not installed or imported.")
        except Exception as e:
            print(f"Error reading or parsing YAML file ({data_yaml_path}): {e}")

        if class_names: # Only proceed if class names were loaded
            print("\nPer-class Mask AP50:")
            per_class_ap50 = metrics.ap50 # List/array of per-class AP50 values
            initial_metrics_summary['Per-Class Mask AP50'] = {}
            for i, idx in enumerate(metrics.ap_class_index):
                if idx < len(class_names):
                    class_name = class_names[idx]
                    # Ensure index 'i' is valid for the per_class_ap50 list/array
                    if i < len(per_class_ap50):
                        ap50_val = per_class_ap50[i]
                        initial_metrics_summary['Per-Class Mask AP50'][class_name] = ap50_val
                        print(f"- {class_name}: {ap50_val:.4f}")
                    else:
                        # This case should be rare if lengths match check passes
                        print(f"- {class_name}: AP50 value not found at index {i}")
                else:
                     print(f"- Class Index {idx}: Invalid index ({idx}) for loaded class names (length {len(class_names)}).")
        else:
           print("\nSkipping per-class metrics display as class names could not be loaded.")

    else:
        print("\nPer-class Mask AP50 metric attributes (ap_class_index, ap50) not available, empty, or structure differs.")
        print(f"  metrics.ap_class_index type: {type(metrics.ap_class_index if hasattr(metrics, 'ap_class_index') else None)}")
        print(f"  metrics.ap50 type: {type(metrics.ap50 if hasattr(metrics, 'ap50') else None)}")

else:
    print("Segmentation metrics object (.seg or .mask) not found or lacks expected attributes (map, map50, map75) in validation results.")
    print("Check 'initial_val_results' attributes using dir(initial_val_results):")
    try:
        print(dir(initial_val_results))
    except NameError:
        print("'initial_val_results' object not defined.")

print("\nInitial metrics stored in 'initial_metrics_summary'")

In [ ]:
# Visualize some predictions on test images from the INITIAL model
def visualize_predictions(model, dataset_path, num_samples=3, model_name="Initial Model"):
    # Get test image paths
    test_img_dir = dataset_path / "test" / "images"
    test_images = [str(test_img_dir / f) for f in os.listdir(test_img_dir)
                   if f.endswith(('.jpg', '.jpeg', '.png'))]

    if len(test_images) == 0:
        print("No test images found. Using validation images instead.")
        test_img_dir = dataset_path / "valid" / "images"
        test_images = [str(test_img_dir / f) for f in os.listdir(test_img_dir)
                       if f.endswith(('.jpg', '.jpeg', '.png'))]

    # Select random samples
    sample_images = random.sample(test_images, min(num_samples, len(test_images)))

    # Make predictions and display results
    for img_path in sample_images:
        # Make prediction
        results = model.predict(img_path, conf=0.5, show=False, save=False)

        # Get the result plot
        for r in results:
            boxes = r.boxes  # Boxes object for bounding box outputs
            masks = r.masks  # Masks object for segmentation masks outputs
            probs = r.probs  # Class probabilities for classification outputs

            # Plot results
            fig, ax = plt.subplots(1, 1, figsize=(12, 9))
            img = r.plot()
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            ax.set_title(f"{model_name} Predictions on {os.path.basename(img_path)}")
            ax.axis('off')
            plt.tight_layout()
            plt.show()

            # Print prediction details
            print(f"\n{model_name} Detections in {os.path.basename(img_path)}:")
            if masks and len(masks) > 0:
                print(f"Total masks: {len(masks)}")
            else:
                print("No masks detected.")

            if boxes and len(boxes) > 0:
                # Get class names from data.yaml
                with open(dataset_path / "data.yaml", 'r') as f:
                    data_cfg = yaml.safe_load(f)
                    class_names = data_cfg['names']

                for i, box in enumerate(boxes):
                    cls_id = int(box.cls.item())
                    cls_name = class_names[cls_id]
                    conf = box.conf.item()
                    print(f"  {i+1}. {cls_name} (Confidence: {conf:.2f})")

# Visualize predictions on test images using the initial model
print("--- Visualizing Initial Model Predictions ---")
visualize_predictions(initial_trained_model, dataset_path, num_samples=3, model_name="Initial Model")

## 4. Fine-Tuning

Now we take the best model from the initial training phase and fine-tune it further. This typically involves training for more epochs with a lower learning rate to allow the model to refine its weights on the specific dataset.

We will:
1. Load the best weights from the initial training.
2. Set fine-tuning hyperparameters (lower learning rate, potentially more epochs).
3. Run the training process again.
4. Evaluate the fine-tuned model and compare its performance.

In [ ]:
# --- 4.1 Load Best Initial Model for Fine-Tuning ---
from ultralytics import YOLO
from pathlib import Path
import torch
import os

# Ensure the path to the best initial model is available
try:
    print(f"Loading model for fine-tuning from: {initial_model_to_load}")
    # Load the model using the path determined during initial evaluation
    model_to_finetune = YOLO(initial_model_to_load)
    print("Model loaded successfully for fine-tuning.")
except NameError:
    print("Error: 'initial_model_to_load' variable not defined.")
    print("Please ensure the initial evaluation cell (93782316) has run successfully.")
    # Optionally, you could try to re-determine the path here, but it's better to rely on the previous step
    # raise Exception("Cannot proceed without the initial model path.")
except Exception as e:
    print(f"An unexpected error occurred loading the model: {e}")
    # raise Exception("Cannot proceed due to model loading error.")

In [ ]:
# --- 4.2 Configure and Run Fine-Tuning ---

# Define a new directory/name for the fine-tuning run
finetune_run_name = "weed_segmentation_finetuned"
print(f"Fine-tuning run will be saved under: {project_dir / finetune_run_name}")

# Fine-tuning parameters
finetune_epochs = 20  # Train for additional epochs
finetune_lr = 0.0001 # Use a lower learning rate (e.g., 1/10th of initial lr0)
finetune_patience = 10 # Patience for early stopping during fine-tuning

# Determine device setting (reuse logic from initial training)
if torch.cuda.is_available() and torch.cuda.device_count() >= 2:
    device_setting = [0, 1]
    batch_size = 16 # Or 16 if memory allows
else:
    device_setting = 0 if torch.cuda.is_available() else 'cpu'
    batch_size = 16 # Adjusted batch size for fine-tuning
print(f"Using device: {device_setting}, Batch size: {batch_size} for fine-tuning")

try:
    print("Starting fine-tuning...")
    finetune_results = model_to_finetune.train(
        data=str(data_yaml_path),  # Use the same (augmented) data config
        epochs=finetune_epochs,
        imgsz=640,
        batch=batch_size,
        patience=finetune_patience,
        device=device_setting,
        project=str(project_dir),
        name=finetune_run_name,    # Save to the new run name
        pretrained=False,          # Start from the loaded weights (initial_model_to_load)
        optimizer="Adam",        # Can keep the same optimizer or switch
        lr0=finetune_lr,           # Use the lower learning rate
        # Other parameters can often be kept the same or adjusted slightly
        single_cls=False,
        dropout=0.0,
        augment=True,              # Keep augmentation enabled
        cache=False,               # Or 'ram'
        verbose=True,
        amp=True,
        close_mosaic=10,           # Turn off mosaic augmentation for the last few epochs of fine-tuning
        cos_lr=True,
        overlap_mask=True,
        mask_ratio=4,
        task='segment',
        workers=4
    )
    print("Fine-tuning training function called.")
except NameError as e:
    print(f"Error starting fine-tuning: A required variable is missing: {e.name}")
    print("Ensure 'model_to_finetune', 'data_yaml_path', 'project_dir' are defined.")
except Exception as e:
    print(f"An unexpected error occurred during fine-tuning: {e}")

### 4.3 Evaluate Fine-Tuned Model

In [ ]:
# --- 4.3 Evaluate the Fine-Tuned Model ---
from ultralytics import YOLO
from pathlib import Path
import yaml
import pandas as pd

# Path to the fine-tuning output directory
finetune_output_dir = project_dir / finetune_run_name
print(f"Looking for fine-tuned results in: {finetune_output_dir}")

# Find the best weights from the fine-tuning run
finetune_weights_dir = finetune_output_dir / "weights"
finetuned_model_path = None
if finetune_weights_dir.exists():
    best_ft_path = finetune_weights_dir / "best.pt"
    last_ft_path = finetune_weights_dir / "last.pt"
    if best_ft_path.exists():
        finetuned_model_path = best_ft_path
        print(f"Found best fine-tuned model: {finetuned_model_path}")
    elif last_ft_path.exists():
        finetuned_model_path = last_ft_path
        print(f"WARNING: best.pt not found in {finetune_weights_dir}. Using last fine-tuned model: {finetuned_model_path}")
    else:
        print(f"Error: Could not find 'best.pt' or 'last.pt' in {finetune_weights_dir}")
else:
    print(f"Error: Weights directory not found for fine-tuning run: {finetune_weights_dir}")

# Proceed only if a fine-tuned model was found
finetuned_metrics_summary = {}
if finetuned_model_path:
    print(f"\nLoading fine-tuned model from: {finetuned_model_path}")
    try:
        finetuned_model = YOLO(finetuned_model_path)

        # Validate the fine-tuned model
        print(f"Validating fine-tuned model using data: {data_yaml_path}")
        finetuned_val_results = finetuned_model.val(data=str(data_yaml_path), task='segment', imgsz=640, batch=8)

        # Display and Store Fine-Tuned Validation Metrics
        print("\nFine-Tuned Validation Metrics (Segmentation):")

        # Access Segmentation Metrics
        ft_metrics = None
        if hasattr(finetuned_val_results, 'seg'):
            ft_metrics = finetuned_val_results.seg
        elif hasattr(finetuned_val_results, 'mask'):
            ft_metrics = finetuned_val_results.mask

        # Print and Store Overall Metrics
        if ft_metrics and hasattr(ft_metrics, 'map') and hasattr(ft_metrics, 'map50') and hasattr(ft_metrics, 'map75'):
            finetuned_metrics_summary['Mask mAP50-95'] = ft_metrics.map
            finetuned_metrics_summary['Mask mAP50'] = ft_metrics.map50
            finetuned_metrics_summary['Mask mAP75'] = ft_metrics.map75
            print(f"Mask mAP50-95: {ft_metrics.map:.4f}")
            print(f"Mask mAP50: {ft_metrics.map50:.4f}")
            print(f"Mask mAP75: {ft_metrics.map75:.4f}")

            # Print and Store Box Precision/Recall
            ft_box_metrics = None
            if hasattr(finetuned_val_results, 'box'):
                ft_box_metrics = finetuned_val_results.box

            if ft_box_metrics and hasattr(ft_box_metrics, 'p') and hasattr(ft_box_metrics, 'r') and \
               hasattr(ft_box_metrics.p, '__len__') and len(ft_box_metrics.p) > 0 and \
               hasattr(ft_box_metrics.r, '__len__') and len(ft_box_metrics.r) > 0:
                 finetuned_metrics_summary['Box Precision'] = ft_box_metrics.p[0]
                 finetuned_metrics_summary['Box Recall'] = ft_box_metrics.r[0]
                 print(f"Box Precision: {ft_box_metrics.p[0]:.4f}")
                 print(f"Box Recall: {ft_box_metrics.r[0]:.4f}")
            elif ft_box_metrics and hasattr(ft_box_metrics, 'p') and hasattr(ft_box_metrics, 'r'):
                 try:
                      finetuned_metrics_summary['Box Precision'] = float(ft_box_metrics.p)
                      finetuned_metrics_summary['Box Recall'] = float(ft_box_metrics.r)
                      print(f"Box Precision: {float(ft_box_metrics.p):.4f}")
                      print(f"Box Recall: {float(ft_box_metrics.r):.4f}")
                 except TypeError:
                       print("Fine-tuned Box precision/recall metrics found but could not be converted to float.")
            else:
                 print("Fine-tuned Box precision/recall metrics not found or structure differs.")

            # Display Per-Class Mask Metrics
            if hasattr(ft_metrics, 'ap_class_index') and hasattr(ft_metrics, 'ap50') and \
               ft_metrics.ap_class_index is not None and ft_metrics.ap50 is not None and \
               hasattr(ft_metrics.ap_class_index, '__len__') and len(ft_metrics.ap_class_index) > 0 and \
               hasattr(ft_metrics.ap50, '__len__') and len(ft_metrics.ap50) == len(ft_metrics.ap_class_index):

                class_names = []
                try:
                    with open(data_yaml_path, 'r') as f:
                        data_config = yaml.safe_load(f)
                        if 'names' in data_config and isinstance(data_config['names'], list):
                            class_names = data_config['names']
                except Exception as e:
                    print(f"Error loading class names from {data_yaml_path}: {e}")

                if class_names:
                    print("\nFine-Tuned Per-class Mask AP50:")
                    ft_per_class_ap50 = ft_metrics.ap50
                    finetuned_metrics_summary['Per-Class Mask AP50'] = {}
                    for i, idx in enumerate(ft_metrics.ap_class_index):
                        if idx < len(class_names):
                            class_name = class_names[idx]
                            if i < len(ft_per_class_ap50):
                                ap50_val = ft_per_class_ap50[i]
                                finetuned_metrics_summary['Per-Class Mask AP50'][class_name] = ap50_val
                                print(f"- {class_name}: {ap50_val:.4f}")
                            else:
                                print(f"- {class_name}: AP50 value not found at index {i}")
                        else:
                             print(f"- Class Index {idx}: Invalid index ({idx}) for loaded class names (length {len(class_names)}).")
                else:
                   print("\nSkipping fine-tuned per-class metrics display as class names could not be loaded.")
            else:
                print("\nFine-tuned Per-class Mask AP50 metric attributes not available or structure differs.")
        else:
            print("Fine-tuned Segmentation metrics object not found or lacks expected attributes.")

        print("\nFine-tuned metrics stored in 'finetuned_metrics_summary'")

    except NameError as e:
        print(f"Error during fine-tuned evaluation: Missing variable {e.name}")
    except FileNotFoundError:
        print(f"Error: Fine-tuned model file not found at {finetuned_model_path}")
    except Exception as e:
        print(f"An unexpected error occurred during fine-tuned evaluation: {e}")
else:
    print("Skipping fine-tuned evaluation as model path was not found.")

In [ ]:
# --- 4.4 Plot Fine-Tuning Training Results ---
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Use the fine-tuning output directory
finetune_output_dir = project_dir / finetune_run_name
print(f"Looking for fine-tuning results.csv in: {finetune_output_dir}")

results_file_candidates = list(finetune_output_dir.glob("results.csv"))

if results_file_candidates:
    results_file = results_file_candidates[0]
    print(f"Loading fine-tuning results from: {results_file}")
    try:
        results_df = pd.read_csv(results_file)
        results_df.columns = results_df.columns.str.strip()

        # Plot key metrics (similar structure to initial plotting)
        plt.figure(figsize=(15, 10))
        epoch_col = 'epoch'
        if epoch_col not in results_df.columns:
             raise ValueError(f"Epoch column ('{epoch_col}') not found in results file.")

        # Plot Losses
        plt.subplot(2, 2, 1)
        loss_cols = ['train/box_loss', 'val/box_loss', 'train/seg_loss', 'val/seg_loss',
                     'train/cls_loss', 'val/cls_loss', 'train/dfl_loss', 'val/dfl_loss']
        plot_added = False
        for col in loss_cols:
            if col in results_df.columns:
                plt.plot(results_df[epoch_col], results_df[col], label=col.replace('/', ' ').replace('_', ' ').title())
                plot_added = True
        if plot_added:
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.title('Fine-Tuning Training and Validation Losses')
            plt.legend()
            plt.grid(True)
        else:
            plt.text(0.5, 0.5, 'No Loss columns found', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        # Plot mAP
        plt.subplot(2, 2, 2)
        map_cols = ['metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/mAP50(M)', 'metrics/mAP50-95(M)']
        plot_added = False
        for col in map_cols:
             if col in results_df.columns:
                label = col.replace('metrics/', '').replace('(B)', ' Box').replace('(M)', ' Mask')
                plt.plot(results_df[epoch_col], results_df[col], label=label)
                plot_added = True
        if plot_added:
            plt.xlabel('Epoch')
            plt.ylabel('mAP')
            plt.title('Fine-Tuning Mean Average Precision (Box and Mask)')
            plt.legend()
            plt.grid(True)
        else:
             plt.text(0.5, 0.5, 'No mAP columns found', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        # Plot Precision/Recall
        plt.subplot(2, 2, 3)
        pr_cols = ['metrics/precision(B)', 'metrics/recall(B)']
        plot_added = False
        for col in pr_cols:
             if col in results_df.columns:
                label = col.replace('metrics/', '').replace('(B)', ' Box').replace('(M)', ' Mask')
                plt.plot(results_df[epoch_col], results_df[col], label=label)
                plot_added = True
        if plot_added:
            plt.xlabel('Epoch')
            plt.ylabel('Value')
            plt.title('Fine-Tuning Precision and Recall')
            plt.legend()
            plt.grid(True)
        else:
            plt.text(0.5, 0.5, 'No Precision/Recall columns found', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        # Plot Learning Rate
        plt.subplot(2, 2, 4)
        lr_cols = [col for col in results_df.columns if col.startswith('lr/') or col.startswith('train/lr')]
        plot_added = False
        if lr_cols:
             for lr_col in lr_cols:
                 if pd.api.types.is_numeric_dtype(results_df[lr_col]):
                     plt.plot(results_df[epoch_col], results_df[lr_col], label=f'{lr_col.split("/")[-1].upper()}')
                     plot_added = True
        if plot_added:
            plt.xlabel('Epoch')
            plt.ylabel('Learning Rate')
            plt.title('Fine-Tuning Learning Rate Schedule')
            plt.legend()
            plt.grid(True)
        else:
             plt.text(0.5, 0.5, 'No Learning Rate columns found', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

        plt.tight_layout()
        plot_save_path = finetune_output_dir / "finetuning_metrics.png"
        plt.savefig(plot_save_path, dpi=300)
        print(f"Fine-tuning metrics plot saved to: {plot_save_path}")
        plt.show()

    except FileNotFoundError:
         print(f"Error: Fine-tuning results file not found at {results_file}")
    except ValueError as e:
         print(f"Error processing fine-tuning results file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while plotting fine-tuning results: {e}")

else:
    print(f"Results file 'results.csv' not found in {finetune_output_dir}. Fine-tuning metrics cannot be plotted.")


In [ ]:
# --- 4.5 Visualize Fine-Tuned Model Predictions ---

# Reuse the visualization function defined earlier
print("--- Visualizing Fine-Tuned Model Predictions ---")
try:
    # Ensure the fine-tuned model object exists
    if 'finetuned_model' in locals() and finetuned_model is not None:
        visualize_predictions(finetuned_model, dataset_path, num_samples=3, model_name="Fine-Tuned Model")
    else:
        print("Fine-tuned model object ('finetuned_model') not found or is None. Cannot visualize predictions.")
        print("Attempting to load from path...")
        if finetuned_model_path and finetuned_model_path.exists():
             print(f"Loading model from {finetuned_model_path}")
             finetuned_model_viz = YOLO(finetuned_model_path)
             visualize_predictions(finetuned_model_viz, dataset_path, num_samples=3, model_name="Fine-Tuned Model")
        else:
             print("Fine-tuned model path not found or invalid. Skipping visualization.")
except NameError:
    print("Error: 'visualize_predictions' function or 'finetuned_model'/'dataset_path' not defined.")
except Exception as e:
    print(f"An error occurred during fine-tuned visualization: {e}")

## 5. Comparison: Initial vs. Fine-Tuned Model

In [ ]:
# --- 5.1 Compare Metrics ---
import pandas as pd

print("--- Comparing Initial vs. Fine-Tuned Metrics ---")

try:
    # Check if both metric dictionaries exist
    if 'initial_metrics_summary' in locals() and 'finetuned_metrics_summary' in locals() and \
       initial_metrics_summary and finetuned_metrics_summary:

        # Create DataFrames for easier comparison
        # Handle potential missing keys gracefully by aligning
        initial_df = pd.DataFrame.from_dict(initial_metrics_summary, orient='index', columns=['Initial'])
        finetuned_df = pd.DataFrame.from_dict(finetuned_metrics_summary, orient='index', columns=['Fine-Tuned'])

        # Separate per-class metrics if they exist
        initial_per_class = initial_df.loc[['Per-Class Mask AP50']] if 'Per-Class Mask AP50' in initial_df.index else None
        finetuned_per_class = finetuned_df.loc[['Per-Class Mask AP50']] if 'Per-Class Mask AP50' in finetuned_df.index else None

        # Drop per-class from main comparison for now
        initial_df = initial_df.drop('Per-Class Mask AP50', errors='ignore')
        finetuned_df = finetuned_df.drop('Per-Class Mask AP50', errors='ignore')

        # Combine overall metrics
        comparison_df = pd.concat([initial_df, finetuned_df], axis=1, sort=True)
        comparison_df = comparison_df.fillna('N/A') # Fill missing metrics

        # Calculate difference for numeric columns
        numeric_cols = comparison_df.select_dtypes(include='number').columns
        if 'Initial' in numeric_cols and 'Fine-Tuned' in numeric_cols:
             comparison_df['Difference'] = comparison_df['Fine-Tuned'] - comparison_df['Initial']
             comparison_df['Difference'] = comparison_df['Difference'].map('{:+.4f}'.format)

        # Format numeric columns
        for col in ['Initial', 'Fine-Tuned']:
             if col in comparison_df.columns:
                 comparison_df[col] = comparison_df[col].apply(lambda x: f"{x:.4f}" if isinstance(x, (int, float)) else x)

        print("\nOverall Metrics Comparison:")
        print(comparison_df.to_string())

        # Compare Per-Class Metrics if available
        if initial_per_class is not None and finetuned_per_class is not None:
            print("\nPer-Class Mask AP50 Comparison:")
            # Extract the dictionaries
            initial_pc_dict = initial_per_class.iloc[0]['Initial'] if not initial_per_class.empty else {}
            finetuned_pc_dict = finetuned_per_class.iloc[0]['Fine-Tuned'] if not finetuned_per_class.empty else {}

            if isinstance(initial_pc_dict, dict) and isinstance(finetuned_pc_dict, dict):
                pc_comp_df = pd.DataFrame([initial_pc_dict, finetuned_pc_dict], index=['Initial', 'Fine-Tuned']).T
                pc_comp_df = pc_comp_df.fillna(0.0)
                pc_comp_df['Difference'] = pc_comp_df['Fine-Tuned'] - pc_comp_df['Initial']
                # Format
                pc_comp_df['Initial'] = pc_comp_df['Initial'].map('{:.4f}'.format)
                pc_comp_df['Fine-Tuned'] = pc_comp_df['Fine-Tuned'].map('{:.4f}'.format)
                pc_comp_df['Difference'] = pc_comp_df['Difference'].map('{:+.4f}'.format)
                print(pc_comp_df.to_string())
            else:
                print("Could not format per-class metrics for comparison (expected dictionaries).")
        else:
            print("\nPer-class metrics not available for both models to compare.")

    else:
        print("Metrics dictionaries ('initial_metrics_summary' or 'finetuned_metrics_summary') not found or empty.")
        print("Ensure both evaluation steps ran successfully and stored results.")

except NameError as e:
    print(f"Error comparing metrics: Missing variable {e.name}")
except Exception as e:
    print(f"An unexpected error occurred during metric comparison: {e}")

In [ ]:
# --- 5.2 Compare Visual Predictions Side-by-Side ---
import matplotlib.pyplot as plt
import cv2
import random
import os
from pathlib import Path
from ultralytics import YOLO

def visualize_comparison(initial_model, finetuned_model, dataset_path, num_samples=3, conf_threshold=0.5):
    print(f"\n--- Visual Comparison (Confidence Threshold: {conf_threshold}) ---")
    # Get test image paths
    test_img_dir = dataset_path / "test" / "images"
    test_images = [str(test_img_dir / f) for f in os.listdir(test_img_dir)
                   if f.endswith(('.jpg', '.jpeg', '.png'))]

    if not test_images:
        print("No test images found. Using validation images instead.")
        test_img_dir = dataset_path / "valid" / "images"
        test_images = [str(test_img_dir / f) for f in os.listdir(test_img_dir)
                       if f.endswith(('.jpg', '.jpeg', '.png'))]

    if not test_images:
        print("No images found in test or validation directories. Cannot visualize.")
        return

    # Select random samples
    sample_images = random.sample(test_images, min(num_samples, len(test_images)))

    for img_path in sample_images:
        print(f"\nProcessing image: {os.path.basename(img_path)}")
        try:
            # Predict with initial model
            initial_results = initial_model.predict(img_path, conf=conf_threshold, verbose=False)
            initial_plot = initial_results[0].plot() # Get annotated image

            # Predict with fine-tuned model
            finetuned_results = finetuned_model.predict(img_path, conf=conf_threshold, verbose=False)
            finetuned_plot = finetuned_results[0].plot() # Get annotated image

            # Display side-by-side
            fig, axes = plt.subplots(1, 2, figsize=(20, 10))

            axes[0].imshow(cv2.cvtColor(initial_plot, cv2.COLOR_BGR2RGB))
            axes[0].set_title(f"Initial Model Predictions")
            axes[0].axis('off')

            axes[1].imshow(cv2.cvtColor(finetuned_plot, cv2.COLOR_BGR2RGB))
            axes[1].set_title(f"Fine-Tuned Model Predictions")
            axes[1].axis('off')

            plt.suptitle(f"Comparison on: {os.path.basename(img_path)}", fontsize=16)
            plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust layout to prevent title overlap
            plt.show()

        except Exception as e:
            print(f"Error processing or plotting image {img_path}: {e}")

# --- Load models if they aren't in memory ---
try:
    if 'initial_trained_model' not in locals() or initial_trained_model is None:
        print("Reloading initial model for comparison...")
        if 'initial_model_to_load' in locals() and initial_model_to_load.exists():
            initial_trained_model = YOLO(initial_model_to_load)
        else:
            raise NameError("Initial model path ('initial_model_to_load') not found or invalid.")

    if 'finetuned_model' not in locals() or finetuned_model is None:
        print("Reloading fine-tuned model for comparison...")
        if 'finetuned_model_path' in locals() and finetuned_model_path.exists():
            finetuned_model = YOLO(finetuned_model_path)
        else:
            raise NameError("Fine-tuned model path ('finetuned_model_path') not found or invalid.")

    # Run the comparison visualization
    visualize_comparison(initial_trained_model, finetuned_model, dataset_path, num_samples=3)

except NameError as e:
    print(f"Error setting up comparison visualization: Missing variable or model: {e}")
    print("Ensure both initial and fine-tuning evaluations ran and model paths are correct.")
except FileNotFoundError as e:
     print(f"Error loading model for comparison: {e}")
except Exception as e:
    print(f"An unexpected error occurred during comparison visualization setup: {e}")

In [ ]:
# This cell can be used for further experiments or analysis

## 6. Export Best Models for Deployment

This section exports the best performing models (initial and fine-tuned) to the ONNX (Open Neural Network Exchange) format. ONNX is a standard format that allows models to be used across different frameworks and deployment environments, including web applications (e.g., using ONNX Runtime for JavaScript).

In [ ]:
# --- 6.1 Export Models to ONNX ---
from ultralytics import YOLO
from pathlib import Path
import os

# Define export directory within the main project directory
export_dir = project_dir / "exported_models"
os.makedirs(export_dir, exist_ok=True)
print(f"Models will be exported to: {export_dir}")

# --- Export Initial Best Model ---
print("\n--- Exporting Initial Model ---")
try:
    # Check if the initial model path variable exists and the file is present
    if 'initial_model_to_load' in locals() and initial_model_to_load and initial_model_to_load.exists():
        print(f"Loading initial model from: {initial_model_to_load}")
        initial_export_model = YOLO(initial_model_to_load)
        
        # Define export path (YOLO export often saves in a subdirectory)
        # The actual saved path might be slightly different (e.g., include _openvino)
        initial_onnx_expected_name = f"{initial_model_to_load.stem}.onnx"
        print(f"Attempting to export initial model to ONNX format (expected name: {initial_onnx_expected_name}) in {export_dir}")
        
        # Export using the model's export method
        # It usually saves the file in the model's run directory or a subfolder
        # We will copy/move it later if needed, or just note the location.
        # Setting export path directly might not work as expected with YOLO's export.
        export_result_path = initial_export_model.export(format='onnx', imgsz=640, opset=12) # Specify imgsz and opset
        print(f"Initial model export process completed. Exported file path: {export_result_path}")
        
        # Verify export
        if Path(export_result_path).exists() and Path(export_result_path).suffix == '.onnx':
             print(f"Verified ONNX file exists: {export_result_path}")
             # Optional: Move the file to the central export_dir if desired
             import shutil
             target_path = export_dir / Path(export_result_path).name
             try:
                # Use copy instead of move to avoid issues if source is needed elsewhere
                shutil.copy(export_result_path, target_path)
                print(f"Copied ONNX file to: {target_path}")
             except Exception as move_e:
                print(f"Could not copy ONNX file: {move_e}")
        else:
             print(f"Warning: Export path {export_result_path} does not seem to be a valid ONNX file. Please check the output location manually.")
                 
    else:
        print("Initial model path ('initial_model_to_load') not found or invalid. Skipping export.")

except NameError as e:
    print(f"Error exporting initial model: Missing variable {e.name}. Ensure previous cells ran.")
except FileNotFoundError as e:
    print(f"Error loading initial model for export: {e}")
except Exception as e:
    print(f"An unexpected error occurred during initial model export: {e}")

# --- Export Fine-Tuned Best Model ---
print("\n--- Exporting Fine-Tuned Model ---")
try:
    # Check if the fine-tuned model path variable exists and the file is present
    if 'finetuned_model_path' in locals() and finetuned_model_path and finetuned_model_path.exists():
        print(f"Loading fine-tuned model from: {finetuned_model_path}")
        finetuned_export_model = YOLO(finetuned_model_path)
        
        # Define export path
        finetuned_onnx_expected_name = f"{finetuned_model_path.stem}.onnx"
        print(f"Attempting to export fine-tuned model to ONNX format (expected name: {finetuned_onnx_expected_name}) in {export_dir}")
        
        # Export
        export_result_path_ft = finetuned_export_model.export(format='onnx', imgsz=640, opset=12)
        print(f"Fine-tuned model export process completed. Exported file path: {export_result_path_ft}")
        
        # Verify export
        if Path(export_result_path_ft).exists() and Path(export_result_path_ft).suffix == '.onnx':
             print(f"Verified ONNX file exists: {export_result_path_ft}")
             # Optional: Move the file
             import shutil
             target_path_ft = export_dir / Path(export_result_path_ft).name
             try:
                # Use copy instead of move
                shutil.copy(export_result_path_ft, target_path_ft)
                print(f"Copied ONNX file to: {target_path_ft}")
             except Exception as move_e:
                print(f"Could not copy ONNX file: {move_e}")
        else:
             print(f"Warning: Export path {export_result_path_ft} does not seem to be a valid ONNX file. Please check the output location manually.")
                 
    else:
        print("Fine-tuned model path ('finetuned_model_path') not found or invalid. Skipping export.")

except NameError as e:
    print(f"Error exporting fine-tuned model: Missing variable {e.name}. Ensure previous cells ran.")
except FileNotFoundError as e:
    print(f"Error loading fine-tuned model for export: {e}")
except Exception as e:
    print(f"An unexpected error occurred during fine-tuned model export: {e}")

print("\n--- Model Export Process Complete ---")

## 7. Comprehensive Model Testing

This section performs inference on a larger batch of images from the test set using the best performing model (preferably the fine-tuned one if available and better). The results (images with predicted masks and boxes) are saved to an output directory for detailed review.

In [ ]:
# --- 7.1 Comprehensive Testing on Test Set ---
import os
import random
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
from tqdm.auto import tqdm

# --- Configuration ---
NUM_TEST_IMAGES = 30  # Number of images to test (set to None to test all)
CONF_THRESHOLD = 0.4  # Confidence threshold for predictions
OUTPUT_SUBDIR = "comprehensive_test_results"

# --- Select Model for Testing ---
model_to_test = None
model_test_path = None
model_test_name = ""

# Prioritize fine-tuned model if it exists and performed well
try:
    if 'finetuned_model_path' in locals() and finetuned_model_path and finetuned_model_path.exists():
        # Optional: Add a check here to compare initial_metrics_summary and finetuned_metrics_summary
        # to decide if the fine-tuned model is actually better.
        # For now, we assume fine-tuned is preferred if it exists.
        model_test_path = finetuned_model_path
        model_test_name = "Fine-Tuned Model"
    elif 'initial_model_to_load' in locals() and initial_model_to_load and initial_model_to_load.exists():
        model_test_path = initial_model_to_load
        model_test_name = "Initial Model"
    else:
        print("Error: No trained model path found ('finetuned_model_path' or 'initial_model_to_load'). Cannot perform testing.")

    if model_test_path:
        print(f"Loading {model_test_name} for comprehensive testing from: {model_test_path}")
        model_to_test = YOLO(model_test_path)
except NameError as e:
    print(f"Error selecting model for testing: Missing variable {e.name}")
except FileNotFoundError as e:
    print(f"Error loading model for testing: {e}")
except Exception as e:
    print(f"An unexpected error occurred loading the model for testing: {e}")

# --- Proceed only if model loaded ---
if model_to_test:
    # --- Prepare Output Directory ---
    test_output_dir = project_dir / OUTPUT_SUBDIR
    os.makedirs(test_output_dir, exist_ok=True)
    print(f"Saving test results to: {test_output_dir}")

    # --- Get Test Images ---
    test_img_dir = dataset_path / "test" / "images"
    all_test_images = [p for p in test_img_dir.glob("*") if p.suffix.lower() in ['.jpg', '.jpeg', '.png']]

    if not all_test_images:
        print("No images found in test set. Checking validation set...")
        test_img_dir = dataset_path / "valid" / "images"
        all_test_images = [p for p in test_img_dir.glob("*") if p.suffix.lower() in ['.jpg', '.jpeg', '.png']]

    if not all_test_images:
        print("Error: No images found in test or validation sets. Cannot perform testing.")
    else:
        print(f"Found {len(all_test_images)} images in {test_img_dir.parent.name} set.")

        # Select subset or all images
        if NUM_TEST_IMAGES is not None and NUM_TEST_IMAGES < len(all_test_images):
            images_to_process = random.sample(all_test_images, NUM_TEST_IMAGES)
            print(f"Processing a random sample of {NUM_TEST_IMAGES} images.")
        else:
            images_to_process = all_test_images
            print(f"Processing all {len(all_test_images)} images.")

        # --- Run Inference and Save Results ---
        print(f"\nRunning inference with confidence threshold: {CONF_THRESHOLD}")
        processed_count = 0
        error_count = 0
        for img_path in tqdm(images_to_process, desc="Testing Images"):
            try:
                # Perform prediction
                results = model_to_test.predict(img_path, conf=CONF_THRESHOLD, verbose=False)

                # Save the plotted result
                if results and results[0].masks is not None: # Check if masks were produced
                    annotated_img = results[0].plot() # Get image with masks/boxes
                    save_filename = f"{img_path.stem}_pred.jpg"
                    save_path = test_output_dir / save_filename
                    cv2.imwrite(str(save_path), annotated_img)
                    processed_count += 1
                else:
                     # Optionally save original image if no detections
                     # print(f"No detections for {img_path.name}")
                     pass

            except Exception as e:
                print(f"\nError processing image {img_path.name}: {e}")
                error_count += 1

        print(f"\n--- Comprehensive Testing Summary ---")
        print(f"Successfully processed and saved results for {processed_count} images.")
        if error_count > 0:
            print(f"Encountered errors processing {error_count} images.")
        print(f"Results saved in: {test_output_dir}")

        # --- Optionally Display a Few Results ---
        num_to_display = min(5, processed_count)
        if num_to_display > 0:
            print(f"\nDisplaying first {num_to_display} saved results:")
            saved_results = list(test_output_dir.glob("*_pred.jpg"))
            for i in range(num_to_display):
                if i < len(saved_results):
                    img = cv2.imread(str(saved_results[i]))
                    plt.figure(figsize=(10, 8))
                    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                    plt.title(saved_results[i].name)
                    plt.axis('off')
                    plt.show()
else:
    print("Skipping comprehensive testing as the model could not be loaded.")